# Data processing for the data needed in the charts

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import numpy as np
from joblib import dump, load


In [2]:
data = pd.read_csv('NewProcedure_Primary_CSV.csv')

## Extracting the data for the Country needed with 2 letter codes

In [3]:
code = "US"

### Collecting the data for OIL PRODUCTION

In [4]:

filtered_data = data[(data['REF_AREA'] == code) & 
                     ((data['ENERGY_PRODUCT'] == 'CRUDEOIL') & (data['FLOW_BREAKDOWN'] == 'INDPROD')) & 
                     (data['UNIT_MEASURE'] == 'KBD')]

# Select columns except for 'ASSESSMENT_CODE'
columns_to_drop = ['ENERGY_PRODUCT', 'FLOW_BREAKDOWN', 'UNIT_MEASURE','ASSESSMENT_CODE']

# Drop the specified columns
filtered_data = filtered_data.drop(columns=columns_to_drop)
filtered_data = filtered_data.rename(columns={'OBS_VALUE': 'PRODUCTION'})

# Save the filtered data to a new CSV file
filtered_data.to_csv(f"PRODUCTION_{code}.csv", index=False)


### Collecting the data for OIL EXPORTS

In [5]:
filtered_data = data[(data['REF_AREA'] == code) & 
                     ((data['ENERGY_PRODUCT'] == 'CRUDEOIL') & (data['FLOW_BREAKDOWN'] == 'TOTEXPSB')) & 
                     (data['UNIT_MEASURE'] == 'KBD')]

# Select columns except for 'ASSESSMENT_CODE'
columns_to_drop = ['ENERGY_PRODUCT', 'FLOW_BREAKDOWN', 'UNIT_MEASURE','ASSESSMENT_CODE']

# Drop the specified columns
filtered_data = filtered_data.drop(columns=columns_to_drop)
filtered_data = filtered_data.rename(columns={'OBS_VALUE': 'EXPORT'})

# Save the filtered data to a new CSV file
filtered_data.to_csv(f"EXPORT_{code}.csv", index=False)


## Combining the EXPORT and PRODUCTION data for a country in a COUNTRY.csv file

In [6]:
df1 = pd.read_csv(f'PRODUCTION_{code}.csv')
df2 = pd.read_csv(f'EXPORT_{code}.csv')

# Extract the column you want to concatenate from the first DataFrame
column_to_concat = df1['PRODUCTION']

# Concatenate the column to the second DataFrame
df2['PRODUCTION'] = column_to_concat

# Save the modified second DataFrame to a new CSV file
df2.to_csv(f'{code}.csv', index=False)

## Coverting the data into JSON for easier use at the backend

In [7]:
# Read the CSV file into a DataFrame
df = pd.read_csv(f'{code}.csv')

# Convert DataFrame to JSON
json_data = df.to_json(orient='records')

# Write JSON data to a file
with open(f'{code}.json', 'w') as f:
    f.write(json_data)


### comobining all the data for the biggest 4 exports grapghs

In [8]:
# Read the data into a DataFrame
code = "US"
df = pd.read_csv(f'{code}.csv')

# Split the TIME_PERIOD column into year and month columns
df[['year', 'month']] = df['TIME_PERIOD'].str.split('-', expand=True)

# Drop the REF_AREA and PRODUCTION columns
df = df.drop(columns=['REF_AREA', 'EXPORT','TIME_PERIOD'])
df = df.rename(columns={'PRODUCTION': 'TARGET'})
# Save the modified DataFrame back to a file
df.to_csv(f"{code}_processed2.csv", index=False)


In [9]:
df1 = pd.read_csv('SA_processed2.csv')
df2 = pd.read_csv('CA_processed2.csv')
df3 = pd.read_csv('MX_processed2.csv')
df4 = pd.read_csv('IQ_processed2.csv')

df1 = df1.rename(columns={'TARGET': 'SA'})
df1['CA'] = df2['TARGET']
df1['MX'] = df3['TARGET']
df1['IQ'] = df4['TARGET']
filtered_data = df1[df1['year'] == 2023]
df1 = df1.drop(columns=['year'])
filtered_data.to_csv(f"DATA2.csv", index=False)


In [10]:
# Read the CSV file into a DataFrame
df = pd.read_csv(f'DATA2.csv')

# Convert DataFrame to JSON
json_data = df.to_json(orient='records')

# Write JSON data to a file
with open(f'DATA2.json', 'w') as f:
    f.write(json_data)


# TRAINING THE ML models

### we got better results when we weighted the newes data with bigger coeficients

In [12]:
code = "SA"

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
# Read the data
data = pd.read_csv(f'{code}_processed.csv')

# Define X and y
X = data[['month']]
y = data['TARGET']

# Define the weight coefficient 'a'
a = 0.04
# Assign weights to the data based on the year
data['weight'] = 1 - (2024 - data['year']) * a

# Instantiate the Random Forest model
random_forest_model = RandomForestRegressor(n_estimators=100, max_depth=10, max_features="sqrt", min_samples_leaf=4, min_samples_split=10, random_state=42)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
random_forest_model.fit(X_train, y_train, sample_weight=data.loc[X_train.index]['weight'])

# Evaluate the model's performance
y_pred = random_forest_model.predict(X_test)

# Print evaluation metrics
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
print("R-squared Score:", r2_score(y_test, y_pred))


Mean Squared Error: 592674.4374702357
R-squared Score: -0.005908907853198642


## Saving the trained modes so it can be used in the backend for prediction

In [17]:
dump(random_forest_model, f'random_forest_model_{code}.joblib')

['random_forest_model_SA.joblib']

#### making an example code so you can predict for a given mounth

In [18]:
the_mounth_you_would_like_to_predict = 3
data_to_predict = pd.DataFrame({'month': [the_mounth_you_would_like_to_predict]})
prediction = random_forest_model.predict(data_to_predict)
float(prediction[0])
# to return a float instead of an one element array

7103.769704099567